In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
from keras.models import load_model
import tensorflow as tf
import numpy as np

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error

Mounted at /content/drive


In [2]:
char_core = ['acc','agr','beta','bm','cash','cashpr','cfp','chatoia','chcsho','chfeps','chinv','chmom',
             'chpmia','chtx','currat','depr','dy','ear','ep','gma','grcapx','grltnoa','ill','indmom','invest',
             'lev','lgr','maxret','mom12m','mom1m','mom36m','mve','nincr','orgcap','pchgm_pchsale','pchsale_pchinvt',
             'pchsale_pchrect','pchsale_pchxsga','retvol','roaq','roavol','roeq','salecash','saleinv','sgr','sp',
             'std_dolvol','std_turn','turn']

core = ['date','permno','ticker','comnam','exchcd','exchname','siccd',
        'indname','size_class','mve_m','ret_adj_ex',]

#core = ['fyear','year','jyear','permno','ticker','comnam','exchcd','exchname','siccd',
#        'indname','size_class','mve_m','rf','ret','ret_adj','ret_ex','ret_adj_ex',]

print(len(char_core), len(core))

data = pd.read_pickle('/content/drive/MyDrive/Colab Notebooks/Dataset/research_result/norm_df49.pkl')
data['date'] = pd.to_datetime(data.index).to_period('M')
data.sort_values(by='date', ascending=True, inplace=True)

X = data[char_core+['date']]
X_info = data[core]
Y = data[['predicted_return','date']]
print(X.shape, Y.shape)

49 11
(210205, 50) (210205, 2)


In [3]:
def my_metric_fn(y_true, y_pred):
    num = tf.reduce_mean(tf.square(y_true - y_pred))
    den = tf.reduce_mean(tf.square(y_true))
    return 1 - num / den

model_file = '/content/drive/MyDrive/Colab Notebooks/Dataset/research_result/model2.h5'

model = load_model(model_file, custom_objects = {"my_metric_fn": my_metric_fn})

In [4]:
import csv
import pickle
import pandas as pd
import numpy as np
import re
import math
import random
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import to_categorical
from keras.regularizers import l1,l2
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
#from utilities import *
#from view import *
import statsmodels.api as sm
import datetime as dt
from dateutil.relativedelta import *
import tensorflow as tf

# comment these if running python instead of ipython
%load_ext autoreload
%matplotlib inline
%autoreload 1

In [73]:
# define neural network model in Keras
def NN(n_inputs, dropout =0.1, l1_reg =0.0, activation='relu', L=4, lr=0.0001):
    # L>0 is the number of hidden layers
    model = Sequential()
    model.add(Dense(units=64, input_dim=n_inputs, kernel_regularizer=l1(l1_reg), activation=activation))
    model.add(Dropout(dropout))
    for i in range (0,L-1):
        model.add(Dense(units=int(32/2**L), kernel_regularizer=l1(l1_reg),  activation=activation))
        model.add(Dropout(dropout))
    model.add(Dense(1))
    opt = tf.keras.optimizers.Adam(learning_rate=lr)
    model.compile(loss='mean_squared_error', optimizer=opt,metrics=['mae'])
    return(model)

In [74]:
Y_2 = Y[Y['date'] >= pd.Period((str(1999)+"-"+str(12)),freq='M')]
X_2 = X[X['date'] >= pd.Period((str(1999)+"-"+str(12)),freq='M')]
Y_2['date'].unique()

<PeriodArray>
['1999-12', '2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06',
 '2000-07', '2000-08', '2000-09',
 ...
 '2021-08', '2021-09', '2021-10', '2021-11', '2021-12', '2022-01', '2022-02',
 '2022-03', '2022-04', '2022-05']
Length: 270, dtype: period[M]

In [76]:
import keras
n_epoches = 100  # maximum number of epoches (to be used with early stopping)
n_batch = 32    # mini-batch size
drop_out = 0.1   # level of dropout (set between 0 and 1)
l1_reg = 0.00  # L_1 regularization parameter
tune = False     # set to true to perform cross-validation for parameter tuning
activation='relu'
y_hat_NN = []
mse_NN = []
mae_NN = []
L=4

es = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=3)

y_real = []
y_hat_NN = []
y_hat_lin = []
for i in range(len(Y_2['date'].unique())):
  train_date = Y_2['date'].unique()[i]
  if train_date == pd.Period((str(2022)+"-"+str(5)),freq='M'):
    break
  test_date = Y_2['date'].unique()[i+1]
  print(test_date)
  x_train = X[X["date"] == train_date ]
  x_train=x_train.drop("date", axis=1)
  y_train = Y[Y["date"] == train_date ]
  y_train= y_train.drop("date", axis=1)
  x_test = X[X["date"] == test_date ]
  x_test=x_test.drop("date", axis=1)
  y_test = Y[Y["date"] == test_date ]
  y_test=y_test.drop("date", axis=1)
  for a in y_test.values.reshape(-1):
    y_real.append(a)

  model = NN(n_inputs=49, dropout=drop_out, l1_reg=l1_reg, activation=activation, L=L)
  model.fit(x_train.values, y_train.values, epochs=n_epoches, batch_size=n_batch, verbose=0, callbacks=[es])
  regr = linear_model.LinearRegression()
  regr.fit(x_train.values, y_train.values)
  yhat_lin = regr.predict(x_test.values).reshape(-1)
  yhat_NN = model.predict(x_test.values).reshape(-1)

  mse, mae = model.evaluate(x_test.values, y_test.values)
  a= np.sum(np.square(yhat_NN - y_test.values.reshape(-1)))
  b = np.sum(np.square(y_test.values.reshape(-1)))
  print("r2:", 1-a/b)
  print("mse:", mse, "mae:",mae)
  print("mean:", np.mean(yhat_NN), "std:", yhat_NN.std())
  for b in yhat_NN:
    y_hat_NN.append(b)
  for c in yhat_lin:
    y_hat_lin.append(c)
  print()

2000-01
Epoch 19: early stopping
15/15 [==============================] - 0s 2ms/step - loss: 0.0527 - mae: 0.1373
r2: 0.03802800741455148
mse: 0.05271519720554352 mae: 0.13725465536117554
mean: -0.008762763 std: 0.03270492

2000-02
Epoch 6: early stopping
15/15 [==============================] - 0s 2ms/step - loss: 0.0409 - mae: 0.1461
r2: -0.04189595062006468
mse: 0.04094763472676277 mae: 0.14607596397399902
mean: -0.010652733 std: 0.017528992

2000-03
Epoch 27: early stopping
14/14 [==============================] - 0s 3ms/step - loss: 0.0214 - mae: 0.1053
r2: -0.024382996811066926
mse: 0.02137102000415325 mae: 0.10529939085245132
mean: 0.023520758 std: 0.013524888

2000-04
Epoch 5: early stopping
14/14 [==============================] - 0s 3ms/step - loss: 0.0178 - mae: 0.1005
r2: -0.003370156127431745
mse: 0.017824510112404823 mae: 0.10048435628414154
mean: 0.0014664765 std: 1.0562057e-05

2000-05
Epoch 18: early stopping
14/14 [==============================] - 0s 3ms/step - loss

In [79]:
print("【Senario 3】MSE of Neural model:",mean_squared_error(y_hat_NN, y_real))
print("【Senario 3】MSE of Linear Regression:",mean_squared_error(y_hat_lin, y_real))

【Senario 3】MSE of Neural model: 0.01753661710319541
【Senario 3】MSE of Linear Regression: 0.018014777658952836


In [66]:
print("【Senario 3】MSE of Neural model:",mean_squared_error(y_hat_NN, y_real))
print("【Senario 3】MSE of Linear Regression:",mean_squared_error(y_hat_lin, y_real))

【Senario 3】MSE of Neural model: 0.02503958607976924
【Senario 3】MSE of Linear Regression: 0.023102333083111555


In [80]:
a = mean_squared_error(y_hat_NN, y_real)
b = mean_squared_error(y_real, [0 for i in range(len(y_real))])
print("【Senario 3】R^2 of Neural model:", 1-a/b )

a = mean_squared_error(y_hat_lin, y_real)
b = mean_squared_error(y_real, [0 for i in range(len(y_real))])
print("【Senario 3】R^2 of Linear REgression:", 1-a/b )

【Senario 3】R^2 of Neural model: -0.1130634150679175
【Senario 3】R^2 of Linear REgression: -0.14341265620206722


----

In [45]:
print("【Senario 3】MSE of Neural model:",mean_squared_error(y_hat, y_real))

【Senario 3】MSE of Neural model: 0.017884938129946862


In [44]:
a = mean_squared_error(y_hat, y_real)
b = mean_squared_error(y_real, [0 for i in range(len(y_real))])
print("【Senario 3】R^2 of Neural model:", 1-a/b )

【Senario 3】R^2 of Neural model: -0.13517163521633413


In [40]:
a=  sum([np.square(a - b) for a, b in zip(y_hat, y_real)])
b = np.square(y_real).sum()
print("r2:", 1-a/b)

r2: -0.1351716352163308


In [39]:
np.square(y_real).sum()

1418.084786760327

In [ ]:
for i in range(len(Y_2['date'].unique())):

  for m in range(1,12):
    print(y,m)
    test_date = pd.Period((str(y)+"-"+str(m)),freq='M')
    train_date = pd.Timestamp(np.datetime64(test_date)).to_pydatetime() + relativedelta(months=-1)
    train_date = train_date.strftime('%Y-%m')

    x_train = X[X["date"] == train_date ]
    x_train=x_train.drop("date", axis=1)
    y_train = Y[Y["date"] == train_date ]
    y_train= y_train.drop("date", axis=1)
    x_test = X[X["date"] == test_date ]
    x_test=x_test.drop("date", axis=1)
    y_test = Y[Y["date"] == test_date ]
    y_test=y_test.drop("date", axis=1)
    if x_test.shape[0] <1:
      break

    model = NN(n_inputs=49, dropout=drop_out, l1_reg=l1_reg, activation=activation, L=L)
    model.fit(x_train.values, y_train.values, epochs=n_epoches, batch_size=n_batch, verbose=0, callbacks=[es])
    yhat = model.predict(x_test.values).reshape(-1)
    mse, mae = model.evaluate(x_test.values, y_test.values)
    a= np.sum(np.square(yhat - y_test.values.reshape(-1)))
    b = np.sum(np.square(y_test.values.reshape(-1)))
    print("r2:", 1-a/b)
    print("mse:", mse, "mae:",mae)
    print("mean:", np.mean(yhat), "std:", yhat.std())
    mse_NN.append(mse)
    mae_NN.append(mae)
    for a in yhat:
      y_hat_NN.append(a)
    print()

In [12]:
len(y_hat_NN)

82603

In [50]:
x_before = X[(X['date']>=pd.Period((str(1999)+"-12"),freq='M')) ]
y_before = Y[(Y['date']>=pd.Period((str(1999)+"-12"),freq='M'))]

In [56]:
y_before['date'].unique()

<PeriodArray>
['1999-12', '2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06',
 '2000-07', '2000-08', '2000-09',
 ...
 '2021-08', '2021-09', '2021-10', '2021-11', '2021-12', '2022-01', '2022-02',
 '2022-03', '2022-04', '2022-05']
Length: 270, dtype: period[M]

In [58]:
for i in range(len(y_before['date'].unique()-1)):
  train_date = y_before['date'].unique()[i]
  test_date = y_before['date'].unique()[i+1]
  print(train_date, test_date)

  x_train = X[X["date"] == train_date ]
  print(x_train.shape)
  x_train=x_train.drop("date", axis=1)
  y_train = Y[Y["date"] == train_date ]
  y_train= y_train.drop("date", axis=1)
  x_test = X[X["date"] == test_date ]
  x_test=x_test.drop("date", axis=1)
  y_test = Y[Y["date"] == test_date ]
  y_test=y_test.drop("date", axis=1)
  if x_train.shape[0] <1:
    break

  model = NN(n_inputs=49, dropout=drop_out, l1_reg=l1_reg, activation=activation, L=L)
  model.fit(x_train.values, y_train.values, epochs=n_epoches, batch_size=n_batch, verbose=0, callbacks=[es])
  yhat = model.predict(x_test.values).reshape(-1)
  mse, mae = model.evaluate(x_test.values, y_test.values)
  a= np.sum(np.square(yhat - y_test.values.reshape(-1)))
  b = np.sum(np.square(y_test.values.reshape(-1)))
  print("r2:", 1-a/b)
  print("mse:", mse, "mae:",mae)
  print("mean:", np.mean(yhat), "std:", yhat.std())
  mse_NN.append(mse)
  mae_NN.append(mae)
  for a in yhat:
    y_hat_NN.append(a)


1999-12 2000-01
(472, 50)
Epoch 16: early stopping


15/15 [==============================] - 0s 2ms/step


15/15 [==============================] - 0s 2ms/step - loss: 0.0533 - mae: 0.1345
r2: 0.02692882769017957
mse: 0.053323425352573395 mae: 0.1344650238752365
mean: -0.031744037 std: 0.039369293
2000-01 2000-02
(471, 50)
Epoch 25: early stopping
15/15 [==============================] - 0s 2ms/step - loss: 0.0412 - mae: 0.1491
r2: -0.048962033242930136
mse: 0.04122534394264221 mae: 0.14910130202770233
mean: 0.0128623955 std: 0.038322568
2000-02 2000-03
(469, 50)
Epoch 16: early stopping
14/14 [==============================] - 0s 3ms/step - loss: 0.0237 - mae: 0.1162
r2: -0.13565430232669606
mse: 0.02369239553809166 mae: 0.11616150289773941
mean: 0.060086295 std: 7.450581e-09
2000-03 2000-04
(442, 50)
Epoch 10: early stopping
14/14 [==============================] - 0s 3ms/step - loss: 0.0181 - mae: 0.1015
r2: -0.01835686285649607
mse: 0.01809074357151985 mae: 0.10150257498025894
mean: 0.0070518875 std: 4.656613e-10
2000-04 2000-05
(441, 50)
Epoch 10: early stopping
14/14 [================

IndexError: ignored

In [60]:
len(y_hat_NN)

172610

In [61]:
y_before

,predicted_return,date
jdate,,
1999-12-31,0.082219,1999-12
1999-12-31,-0.121525,1999-12
1999-12-31,-0.093906,1999-12
1999-12-31,-0.176926,1999-12
1999-12-31,-0.059745,1999-12
...,...,...
2022-05-31,-0.131135,2022-05
2022-05-31,-0.208780,2022-05
2022-05-31,-0.111692,2022-05


In [ ]:
# Linear Model of Senario 2
y_hat_linear = []
mse_linear = []
r2_linear = []

for i in range(2000, 2023):
    print(f"Year {i}")
    x_train = X[ (X['date'] < pd.Period((str(i)+"-1"),freq='M')) ]
    y_train = Y[ (Y['date'] < pd.Period((str(i)+"-1"),freq='M')) ]

    x_test = X[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]
    y_test = Y[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]

    x_train = x_train.drop("date", axis=1)
    y_train = y_train.drop("date", axis=1)
    x_test = x_test.drop("date", axis=1)
    y_test = y_test.drop("date", axis=1)

    regr = linear_model.LinearRegression()
    regr.fit(x_train, y_train)
    y_pred_linear = regr.predict(x_test)
    mse = mean_squared_error(y_test, y_pred_linear)
    for a in y_pred_linear.reshape(-1):
        y_hat_linear.append(a)

In [18]:
X_info = data[core]
x_test_info = X_info[(X_info['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]

x_test = X[(X['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]
y_test = Y[(Y['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]

senario_2_result = x_test_info.copy()
senario_2_result.index = x_test.index

senario_2_result['y_real'] = y_test['predicted_return']
senario_2_result['NN_pred_pretrain'] = y_hat_all
senario_2_result['lin_pred_pretrain'] = y_hat_linear

senario_2_result.to_pickle("senario_2_result1.pkl")

In [19]:
## MSE
mse_Linear = mean_squared_error(senario_2_result['y_real'], senario_2_result['lin_pred_pretrain'])
mse_NN = mean_squared_error(senario_2_result['y_real'], senario_2_result['NN_pred_pretrain'])
print(f"【Senario 2】MSE of Linear model: {mse_Linear}")
print(f"【Senario 2】MSE of Neural model: {mse_NN}")

【Senario 2】MSE of Linear model: 0.01566540810352375
【Senario 2】MSE of Neural model: 0.015703034167274834


In [20]:
# R^2 of Linear Regresion
a = np.sum(np.square(senario_2_result['y_real'] -  senario_2_result['lin_pred_pretrain']))
b = np.sum(np.square(senario_2_result['y_real']))
print(f"【Senario 2】 R^2 of Linear Regresion :",1-a/b)

# R^2 of Neural Network
a = np.sum(np.square(senario_2_result['y_real'] -  senario_2_result['NN_pred_pretrain']))
b = np.sum(np.square(senario_2_result['y_real']))
print(f"【Senario 2】 R^2 of Neural Network :",1-a/b)

【Senario 2】 R^2 of Linear Regresion : 0.0057037489309388345
【Senario 2】 R^2 of Neural Network : 0.003315591923923278


In [21]:
# Metrics: XS-R^2 of Neural Network Model  (weighted)
stock_list = senario_2_result['permno'].unique()
month_list = senario_2_result.index.unique()
print(len(stock_list), len(month_list))

r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = senario_2_result[senario_2_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['NN_pred_pretrain']).sum() / num_month) * num_month
  deno = np.square((df['NN_pred_pretrain']).sum() / num_month)  * num_month
  num_.append(num)
  deno_.append(deno)
print(f"XS-R^2 of Neural Network Model  (weighted):",1 - np.mean(num_) / np.mean(deno_))

# Metrics: XS-R^2 of Linear Regresion (weighted)
r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = senario_2_result[senario_2_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['lin_pred_pretrain']).sum() / num_month) * num_month
  deno = np.square((df['lin_pred_pretrain']).sum() / num_month)  * num_month
  num_.append(num)
  deno_.append(deno)
print(f"XS-R^2 of Linear Regresion (weighted):", 1 - np.mean(num_) / np.mean(deno_))

607 269
XS-R^2 of Neural Network Model  (weighted): -0.7404501948643314
XS-R^2 of Linear Regresion (weighted): -0.9542138628254369


In [24]:
# Metrics: XS-R^2 of Neural Network Model  (no weighted)
stock_list = senario_2_result['permno'].unique()
month_list = senario_2_result.index.unique()
print(len(stock_list), len(month_list))

r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = senario_2_result[senario_2_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['NN_pred_pretrain']).sum() / num_month)
  deno = np.square((df['NN_pred_pretrain']).sum() / num_month)
  num_.append(num)
  deno_.append(deno)
print(f"【Senario 2】 XS-R^2 of Neural Network Model  (no weighted):",1 - np.mean(num_) / np.mean(deno_))

# Metrics: XS-R^2 of Linear Regresion (no weighted)
r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = senario_2_result[senario_2_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['lin_pred_pretrain']).sum() / num_month)
  deno = np.square((df['lin_pred_pretrain']).sum() / num_month)
  num_.append(num)
  deno_.append(deno)
print(f"【Senario 2】 XS-R^2 of Linear Regresion (no weighted):", 1 - np.mean(num_) / np.mean(deno_))

607 269
【Senario 2】 XS-R^2 of Neural Network Model  (no weighted): -19.60326057898728
【Senario 2】 XS-R^2 of Linear Regresion (no weighted): -23.84399135612912


In [22]:
# Metrics: EV of Neural Network
r2_stock = {}
num_ = []
deno_ = []
for i in month_list:
  df = senario_2_result[senario_2_result.index == i]
  num_stock = df.shape[0]
  num = (np.square(df['y_real'] -  df['NN_pred_pretrain'])).sum() / num_stock
  deno = (np.square(df['y_real'])).sum() /num_stock
  num_.append(num)
  deno_.append(deno)
  break
print(f"【Senario 2】 EV of Neural Network:",1 - np.mean(num_) / np.mean(deno_))

# Metrics: EV of Linear Model
r2_stock = {}
num_ = []
deno_ = []
for i in month_list:
  df = senario_2_result[senario_2_result.index == i]
  num_stock = df.shape[0]
  num = (np.square(df['y_real'] -  df['lin_pred_pretrain'])).sum() / num_stock
  deno = (np.square(df['y_real'])).sum() /num_stock
  num_.append(num)
  deno_.append(deno)
  break
print(f"【Senario 2】 EV of Linear Model:",1 - np.mean(num_) / np.mean(deno_))

【Senario 2】 EV of Neural Network: -0.00010379938953031242
【Senario 2】 EV of Linear Model: 0.004059510189341364


In [ ]:
# Neural Network

import keras
y_hat_all = []
mse_ = {}
r2_ = {}

for i in range(2000, 2023):
    print(f"Year {i}")
    x_train = X[ (X['date'] < pd.Period((str(i)+"-1"),freq='M')) ]
    y_train = Y[ (Y['date'] < pd.Period((str(i)+"-1"),freq='M')) ]

    x_test = X[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]
    y_test = Y[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]

    x_train = x_train.drop("date", axis=1)
    y_train = y_train.drop("date", axis=1)
    x_test = x_test.drop("date", axis=1)
    y_test = y_test.drop("date", axis=1)

    model_1yr = load_model(model_file, custom_objects = {"my_metric_fn": my_metric_fn})
    model_1yr.fit(x_train, y_train, epochs=30,
                  callbacks=[keras.callbacks.EarlyStopping(monitor='loss', patience=2)],  batch_size = 256,)
    mse, r2, mae = model_1yr.evaluate(x_test, y_test)
    print(mse, r2, mae)
    mse_[i] =mse
    r2_[i] = r2
    y_NN2 = model_1yr.predict(x_test)
    for a in y_NN2.reshape(-1):
        y_hat_all.append(a)
    print(y_NN2.reshape(-1).mean(), y_NN2.reshape(-1).std())

In [ ]:
# Linear Model of Senario 2
y_hat_linear = []
mse_linear = []
r2_linear = []

for i in range(2000, 2023):
    print(f"Year {i}")
    x_train = X[ (X['date'] < pd.Period((str(i)+"-1"),freq='M')) ]
    y_train = Y[ (Y['date'] < pd.Period((str(i)+"-1"),freq='M')) ]

    x_test = X[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]
    y_test = Y[(X['date'] >= pd.Period((str(i)+"-1"),freq='M')) & (X['date'] <= pd.Period((str(i)+"-12"),freq='M'))]

    x_train = x_train.drop("date", axis=1)
    y_train = y_train.drop("date", axis=1)
    x_test = x_test.drop("date", axis=1)
    y_test = y_test.drop("date", axis=1)

    regr = linear_model.LinearRegression()
    regr.fit(x_train, y_train)
    y_pred_linear = regr.predict(x_test)
    mse = mean_squared_error(y_test, y_pred_linear)
    for a in y_pred_linear.reshape(-1):
        y_hat_linear.append(a)

In [8]:
X_info = data[core]
x_test_info = X_info[(X_info['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]

x_test = X[(X['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]
y_test = Y[(Y['date'] >= pd.Period((str(2000)+"-1"),freq='M'))]

senario_2_result = x_test_info.copy()
senario_2_result.index = x_test.index

senario_2_result['y_real'] = y_test['predicted_return']
senario_2_result['NN_pred_pretrain'] = y_hat_all
senario_2_result['lin_pred_pretrain'] = y_hat_linear

senario_2_result.to_pickle("senario_2_result.pkl")

In [9]:
## MSE
mse_Linear = mean_squared_error(senario_2_result['y_real'], senario_2_result['lin_pred_pretrain'])
mse_NN = mean_squared_error(senario_2_result['y_real'], senario_2_result['NN_pred_pretrain'])
print(f"【Senario 2】MSE of Linear model: {mse_Linear}")
print(f"【Senario 2】MSE of Neural model: {mse_NN}")
print()

# R^2 of Linear Regresion
a = np.sum(np.square(senario_2_result['y_real'] -  senario_2_result['lin_pred_pretrain']))
b = np.sum(np.square(senario_2_result['y_real']))
print(f"【Senario 2】 R^2 of Linear Regresion :",1-a/b)

# R^2 of Neural Network
a = np.sum(np.square(senario_2_result['y_real'] -  senario_2_result['NN_pred_pretrain']))
b = np.sum(np.square(senario_2_result['y_real']))
print(f"【Senario 2】 R^2 of Neural Network :",1-a/b)

【Senario 2】MSE of Linear model: 0.01566540810352375
【Senario 2】MSE of Neural model: 0.0156932345213041


In [10]:
# R^2 of Linear Regresion
a = np.sum(np.square(senario_2_result['y_real'] -  senario_2_result['lin_pred_pretrain']))
b = np.sum(np.square(senario_2_result['y_real']))
print(f"【Senario 2】 R^2 of Linear Regresion :",1-a/b)

# R^2 of Neural Network
a = np.sum(np.square(senario_2_result['y_real'] -  senario_2_result['NN_pred_pretrain']))
b = np.sum(np.square(senario_2_result['y_real']))
print(f"【Senario 2】 R^2 of Neural Network :",1-a/b)

【Senario 2】 R^2 of Linear Regresion : 0.0057037489309388345
【Senario 2】 R^2 of Neural Network : 0.003937583460059013


In [12]:
# Metrics: XS-R^2 of Neural Network Model  (weighted)
stock_list = senario_2_result['permno'].unique()
month_list = senario_2_result.index.unique()
print(len(stock_list), len(month_list))

r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = senario_2_result[senario_2_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['NN_pred_pretrain']).sum() / num_month) * num_month
  deno = np.square((df['NN_pred_pretrain']).sum() / num_month)  * num_month
  num_.append(num)
  deno_.append(deno)
print(f"XS-R^2 of Neural Network Model  (weighted):",1 - np.mean(num_) / np.mean(deno_))

# Metrics: XS-R^2 of Linear Regresion (weighted)
r2_stock = {}
num_ = []
deno_ = []
for i in stock_list:
  df = senario_2_result[senario_2_result['permno'] == i]
  num_month = df.shape[0]
  num = np.square((df['y_real'] -  df['lin_pred_pretrain']).sum() / num_month) * num_month
  deno = np.square((df['lin_pred_pretrain']).sum() / num_month)  * num_month
  num_.append(num)
  deno_.append(deno)
print(f"XS-R^2 of Linear Regresion (weighted):", 1 - np.mean(num_) / np.mean(deno_))

XS-R^2 of Neural Network Model  (weighted): -0.1614102688995509
XS-R^2 of Linear Regresion (weighted): -0.9542138628254369


In [13]:
# Metrics: EV of Neural Network
r2_stock = {}
num_ = []
deno_ = []
for i in month_list:
  df = senario_2_result[senario_2_result.index == i]
  num_stock = df.shape[0]
  num = (np.square(df['y_real'] -  df['NN_pred_pretrain'])).sum() / num_stock
  deno = (np.square(df['y_real'])).sum() /num_stock
  num_.append(num)
  deno_.append(deno)
  break
print(f"【Senario 2】 EV of Neural Network:",1 - np.mean(num_) / np.mean(deno_))

# Metrics: EV of Linear Model
r2_stock = {}
num_ = []
deno_ = []
for i in month_list:
  df = senario_2_result[senario_2_result.index == i]
  num_stock = df.shape[0]
  num = (np.square(df['y_real'] -  df['lin_pred_pretrain'])).sum() / num_stock
  deno = (np.square(df['y_real'])).sum() /num_stock
  num_.append(num)
  deno_.append(deno)
  break
print(f"【Senario 2】 EV of Linear Model:",1 - np.mean(num_) / np.mean(deno_))

【Senario 2】 EV of Neural Network: -0.0011904280992580674
【Senario 2】 EV of Linear Model: 0.004059510189341364
